In [1]:
from llama_index.core.tools import FunctionTool
from pycoingecko import CoinGeckoAPI
from llama_index.llms.groq import Groq
from duckduckgo_search import DDGS
from typing import List, Dict, Union , Optional
import requests
import asyncio
from deepgram import DeepgramClient, SpeakOptions
import os
import json
from datetime import datetime

llm = Groq(model="llama3-groq-70b-8192", api_key="")


In [2]:

def fetch_price(crypto_id: str, vs_currency: str = 'usd') -> Optional[float]:
    """
    This function is to fetch the current price of a specified cryptocurrency using CoinGecko API.

    :param crypto_id: The name of the cryptocurrency. Spaces are replaced with hyphens (e.g., 'shiba inu' -> 'shiba-inu').
    :param vs_currency: The fiat currency to compare against (default is 'usd').
    :return: The current price of the cryptocurrency in the specified fiat currency, or None if the cryptocurrency is not found.
    """
    crypto_id = crypto_id.replace(" ", "-").lower()
    url = "https://api.coingecko.com/api/v3/simple/price"
    headers = {"accept": "application/json"}
    params = {'ids': crypto_id, 'vs_currencies': vs_currency}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        price_data = response.json()
        return price_data.get(crypto_id, {}).get(vs_currency)
    else:
        return None

fetch_price_tool = FunctionTool.from_defaults(fn=fetch_price)

# def internet_search(query: str,region: Optional[str] = "wt-wt",max_results: Optional[int] = 5,) -> List[Dict]:
#     """
#     Only use this function when you do not know the answer, this function is for searching the internet using duckduckgo.

#     Args:
#         query (str): The query to be passed to DuckDuckGo.
#         region (Optional[str]): The region to be used for the search in [country-language] convention, e.g., us-en, uk-en, ru-ru, etc.
#         max_results (Optional[int]): The maximum number of results to be returned.

#     Returns:
#         List[Dict]: A list of dictionaries containing the search results.
#     """
#     params = {
#         "keywords": query,
#         "region": region,
#         "max_results": max_results,
#     }

#     with DDGS() as ddg:
#         return list(ddg.text(**params))

# internet_search_tool = FunctionTool.from_defaults(fn=internet_search)

def bing_news_search(query: str, api_key: str = "1d6de50036de4f71b63896eb7f0b2821", max_results: Optional[int] = 5) -> List[Dict]:
    """
    Search for news using Bing News Search API.

    Args:
        query (str): The query to be passed to Bing News Search.
        api_key (str): The API key for Bing Search.
        max_results (Optional[int]): The maximum number of results to be returned. Defaults to 5.

    Returns:
        List[Dict]: A list of dictionaries containing the news results.
    """
    url = "https://api.bing.microsoft.com/v7.0/news/search"
    headers = {
        "Ocp-Apim-Subscription-Key": api_key
    }
    params = {
        "q": query,
        "count": max_results,
        "freshness": "Day"  
    }
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    
    results = response.json()
    news_articles = results.get("value", [])
    
    formatted_results = [
        {
            "name": article["name"],
            "url": article["url"],
            "snippet": article["description"]
        }
        for article in news_articles
    ]
    
    return formatted_results

internet_search_tool = FunctionTool.from_defaults(fn=bing_news_search)

# def get_and_display_top_pools(top_n: int = 5) -> List[Dict[str, Union[str, int, float]]]:
#     """
#     Retrieve the latest data for all pools from the yields.llama.fi API, including enriched information such as predictions,
#     and return the top pools sorted by APY.

#     :param top_n: The number of top pools to return, sorted by APY.
#     :return: A list of dictionaries containing the top pools' information.
#     """
#     url = "https://yields.llama.fi/pools"
#     headers = {"accept": "application/json"}

#     response = requests.get(url, headers=headers)

#     if response.status_code == 200:
#         pools_data = response.json().get('data', [])
#         sorted_pools = sorted(pools_data, key=lambda x: x['apy'], reverse=True)
#         return sorted_pools[:top_n]
#     else:
#         return [{"Error": f"Failed to retrieve data: {response.status_code}"}]


# get_pool_tool = FunctionTool.from_defaults(fn=get_and_display_top_pools)


# def get_stable_coins(top_m: int = 5) -> List[Dict[str, Union[str, float]]]:
#     """
#     This function retrieves the latest data for stablecoins from the stablecoins.llama.fi API,
#     including their prices, and returns the top stablecoins by market cap.

#     :param top_m: The number of top stablecoins to return.
#     :return: A list of dictionaries containing the top stablecoins' information.
#     """
#     url = "https://stablecoins.llama.fi/stablecoins?includePrices=true"
#     response = requests.get(url)

#     if response.status_code == 200:
#         stablecoins_data = response.json()
#         stablecoins = stablecoins_data["peggedAssets"][:top_m]
#         return stablecoins
#     else:
#         return [{"Error": f"Failed to retrieve data: {response.status_code}"}]

# fetch_stablecoins_tool = FunctionTool.from_defaults(fn=get_stable_coins)


# def get_stable_coin_prices() -> List[Dict[str, Union[str, float]]]:
#     """
#     This function retrieves the latest stablecoin prices from the stablecoins.llama.fi API
#     and returns a list of dictionaries containing the stablecoins' information.

#     :return: A list of dictionaries containing the stablecoins' information.
#     """
#     url = "https://stablecoins.llama.fi/stablecoinprices"
#     response = requests.get(url)

#     if response.status_code == 200:
#         stablecoin_prices = response.json()
#         stablecoin_prices

#         return stablecoin_prices
#     else:
#         return [{"Error": f"Failed to retrieve data: {response.status_code}"}]

# fetch_stable_coin_prices_tool = FunctionTool.from_defaults(fn=get_stable_coin_prices)


def get_fees_and_revenue(data_type: str,
                         exclude_total_data_chart: bool = True,
                         exclude_total_data_chart_breakdown: bool = True,
                         protocol_name: str = None) -> Union[Dict, List[Dict]]:
    """
    This function retrieves fees and revenue data from the llama.fi API.

    :param data_type: Desired data type, this will be given by the user (e.g., "dailyFees", "totalFees", "dailyRevenue", "totalRevenue").
    :param exclude_total_data_chart: Boolean flag to exclude the aggregated chart from the response. Default is True.
    :param exclude_total_data_chart_breakdown: Boolean flag to exclude the broken down chart from the response. Default is True.
    :param protocol_name: Name of the protocol to filter the results by. Default is None (no filtering).
    :return: A dictionary or list containing the raw fees and revenue data.
    """
    url = "https://api.llama.fi/overview/fees"
    params = {
        "excludeTotalDataChart": str(exclude_total_data_chart).lower(),
        "excludeTotalDataChartBreakdown": str(exclude_total_data_chart_breakdown).lower(),
        "dataType": data_type
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if protocol_name:
            if "protocols" in data:
                data["protocols"] = [protocol for protocol in data["protocols"] if protocol_name.lower() in protocol["name"].lower()]
        return data
    else:
        return {"Error": f"Failed to retrieve data: {response.status_code}"}


get_fees_and_revenue_tool = FunctionTool.from_defaults(fn=get_fees_and_revenue)


def get_protocol_tvl_data(protocol_name: Optional[str] = None) -> Union[Dict, List[Dict]]:
    """
    This function retrieves protocols from the llama.fi API and their TVL data.

    :param protocol_name: Optional; The name of the protocol to filter by. If None, returns data for all protocols.
    :return: A dictionary or list containing the raw TVL data, or a single protocol's data if protocol_name is provided.
    """
    url = "https://api.llama.fi/protocols"
    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        protocols = response.json()
        
        if protocol_name:
            for protocol in protocols:
                if protocol["name"].lower() == protocol_name.lower():
                    return protocol
            return {"Error": "Protocol not found"}
        
        return protocols
    
    else:
        return {"Error": f"Failed to retrieve data: {response.status_code}"}


get_tvl_data_tool = FunctionTool.from_defaults(fn=get_protocol_tvl_data)


def get_historicalchain_tvl_data(top_m: int = None) -> Union[Dict, List[Dict]]:
    """
    This function retrieves historical TVL (total value locked) data of DeFi across all chains.
    It excludes liquid staking and double-counted TVL.

    :param top_m: Number of top days to fetch. If None, fetches all available data.
    :return: A list of dictionaries containing TVL data with human-readable dates if the request is successful,
             or a dictionary with an error message if the request fails.
    """
    url = "https://api.llama.fi/v2/historicalChainTvl"
    headers = {"accept": "application/json"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        if top_m is not None:
            top_m = min(top_m, len(data))
            data = data[:top_m]

        for entry in data:
            entry['date'] = datetime.fromtimestamp(entry['date']).strftime('%Y-%m-%d %H:%M:%S')

        return data

    except requests.RequestException as e:
        return {"Error": f"Failed to retrieve data: {str(e)}"}
    

get_historicalchain_tvl_data_tool = FunctionTool.from_defaults(fn=get_historicalchain_tvl_data)

def overview_dexs(data_type: str, dex_name: str,
                           exclude_total_data_chart: bool = True,
                           exclude_total_data_chart_breakdown: bool = True) -> Union[Dict, List[Dict]]:
    """
    This function retrieves data related to a specific DEX along with summaries of their volumes and dataType history data.

    :param data_type: Desired data type, this will be given by the user. (e.g., "dailyVolume", "totalVolume")
    :param dex_name: Name of the DEX to filter the data for. (Example : '0x', '10KSwap', '3xcalibur', '4Swap', 'Arbitrum Exchange V2', 'Arbitrum Exchange V3',
    'DerpDEX', 'FeeFree', 'ICDex', 'MantisSwap', 'Omnidrome', 'Scale', 'SmarDex',
    'Viridian Exchange', 'ABcDeFX', 'Acala Swap', 'Aequinox', 'Aerodrome V1',
    'Aerodrome Slipstream', 'Agni Finance', 'AirSwap', 'Aktionariat', 'Aldrin', 'ALEX',
    'Algofi Swap', 'AlienFi', 'Allbridge Classic', 'Ambient', 'AmpleSwap', 'Ape.Store',
    'ApeSwap AMM', 'ArcherSwap', 'Archly V1', 'Archly V2', 'Ascent Exchange V1',
    'Ascent Exchange V3', 'Ashswap', 'Astroport Classic', 'Astroport', 'AstroSwap',
    'Auragi Finance', 'AuraSwap', 'AuroraSwap', 'AutoShark', 'AUX Exchange', 'Axial',
    'BabyDogeSwap', 'BabySwap', 'Balanced Exchange', 'Balancer V1', 'Balancer V2',
    'Bancor V3', 'Bancor V2.1', 'BaseSwap V2', 'BaseX', 'Basin Exchange', 'Beamex',
    'BeamSwap Classic', 'BeamSwap Stable AMM', 'BeamSwap V3', 'Beethoven X DEX',
    'BenSwap', 'Bisq', 'BiSwap V2', 'BladeSwap AMM', 'BladeSwap CL', 'BLEX', 'Blitz',
    'Blue Planet', 'BMX', 'TanX.fi', 'Butter.xyz', 'BXH', 'Camelot V2', 'Camelot V3',
    'Canary', 'CandySwap', 'Canto Dex', 'Capricorn Finance', 'Capybara Exchange',
    'Carbon Defi', 'CarbonSwap', 'Cauldron', 'CaviarNine Shape Liquidity', 'CaviarNine LSU Pool',
    'Cellana Finance', 'Cetus', 'Chainge Finance', 'ChampagneSwap', 'CherrySwap', 'Chimp Exchange',
    'Chronos V1', 'Chronos V2', 'ClaimSwap V1', 'Clipper', 'Clober V1', 'Clober V2',
    'Cometh', 'Complus Network', 'Concordex', 'Cone', 'Covo V2', 'Crema Finance',
    'CrescentSwap', 'Crodex', 'Cropper CLMM', 'CroSwap', 'CryptoSwap', 'Curve DEX',
    'DackieSwap V3', 'DackieSwap V2', 'Danogo', 'DAO Swap', 'Darkness', 'DeDust',
    'DeepBook', 'Defi Kingdoms', 'Defi Swap', 'DefiBox', 'DefiChain DEX', 'DefiPlaza',
    'DeltaSwap', 'Demex', 'Dexalot', 'dexSWAP', 'Persistence DEX', 'DFX V2', 'Dfyn Network',
    'Dinosaur Eggs', 'DODO', 'Dove Swap V3', 'DPEX', 'DragonSwap V2', 'DragonSwap V3',
    'Dragon Swap', 'Swaap Maker V1', 'Swaap Maker V2', 'SwapBased AMM', 'SwapBased Concentrated Liquidity',
    'Swapline', 'Swappi', 'Swapr V2', 'Swapsicle V2', 'Swop', 'Symmetric', 'SyncSwap',
    'Synthetify', 'Syrup Finance', 'TangleSwap', 'Tealswap V2', 'Tegro', 'TempleDAO Trade',
    'Terraswap', 'Tethys AMM', 'Tetu Swap', 'ThalaSwap', 'Thena V1', 'THENA FUSION', 'Thick',
    'Thorchain', 'Throne V3', 'Tinyman', 'Titano Swych', 'Tokenlon', 'Tomb Swap', 'Trader Joe DEX',
    'Joe V2', 'Joe V2.2', 'Trisolaris', 'TTswap', 'Turbos', 'Ubeswap', 'UltronSwap', 'Unicly',
    'UniFi', 'VanSwap', 'VaporDex V1', 'VaporDex V2', 'Veax', 'Velocimeter V2', 'Velocore V2',
    'Velodrome V1', 'Velodrome V3', 'Velodrome V2', 'VeniceSwap', 'Verse', 'Vertex', 'Vexchange',
    'ViperSwap', 'Voltage AMM', 'Voltage V3', 'VoltSwap V2', 'Voodoo Trade Base', 'VVS Standard',
    'WAGMI', 'WagyuSwap', 'WannaSwap', 'WanSwap Dex', 'WardenSwap', 'WarpGate', 'Wavelength DAO',
    'Web3.world', 'WEMIX Full Range Deposit', 'WhaleSwap', 'WigoSwap', 'WinerySwap', 'WingRiders',
    'WingSwap', 'Wojak Finance', 'Wombat Exchange', 'WOOFi Swap', 'Xei Finance', 'Xena Finance',
    'xExchange', 'Xfai', 'XSwap V2', 'YFX', 'YFX V4', 'YieldFields', 'Yodeswap', 'YokaiSwap',
    'Yoshi Exchange', 'Zebra V1', 'Zebra V2', 'ZigZag', 'ZilSwap', 'ZipSwap', 'Zircon Gamma',
    'zkSwap Finance', 'zkLite Exchange', 'ZKBase', 'Zyberswap AMM', 'Zyberswap V3', 'ZyberSwap Stableswap',
    'Cleopatra CL', 'Junoswap', 'Alien Base AMM', 'Alita Finance', 'ArthSwap V3', 'BitGenie AMM',
    'Blasterswap V3', 'Blasterswap V2', 'Cleopatra Legacy', 'EddyFinance', 'Infusion', 'KIM Exchange V2',
    'KIM Exchange V3', 'Merchant Moe Liquidity Book', 'Moraswap V3', 'Omax Swap', 'PancakeSwap AMM V1',
    'PancakeSwap AMM', 'PancakeSwap StableSwap', 'PancakeSwap AMM V3', 'Pharaoh Legacy', 'SMBSwap V2',
    'SMBSwap V3', 'SwapMode', 'Thruster V2', 'Thruster V3', 'Uniswap V1', 'Uniswap V2', 'Uniswap V3',
    'VanillaSwap V2', 'VanillaSwap V3')
    :param exclude_total_data_chart: Boolean flag to exclude the aggregated chart from the response. Default is True.
    :param exclude_total_data_chart_breakdown: Boolean flag to exclude the broken down chart from the response. Default is True.
    :return: A dictionary containing the summaries of the DEX volumes and dataType history data or an error message.
    """
    url = "https://api.llama.fi/overview/dexs"
    params = {
        "excludeTotalDataChart": str(exclude_total_data_chart).lower(),
        "excludeTotalDataChartBreakdown": str(exclude_total_data_chart_breakdown).lower(),
        "dataType": data_type
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        for protocol in data["protocols"]:
            if protocol["name"] == dex_name:
                return json.dumps(protocol , indent =4)
        return {"Error": f"DEX named {dex_name} not found"}
    else:
        return {"Error": f"Failed to retrieve data: {response.status_code}"}


get_overview_dexs_tool = FunctionTool.from_defaults(fn=overview_dexs)


def overview_options(data_type: str,
                     exclude_total_data_chart: bool = True,
                     exclude_total_data_chart_breakdown: bool = True,
                     dex_name: str = None) -> Union[Dict, List[Dict]]:
    """
    This function retrieves all options DEXs along with summaries of their volumes and dataType history data,
    and filters the results by the DEX name if provided.

    :param data_type: Desired data type, this will be given by the user (e.g., "dailyPremiumVolume", "totalPremiumVolume", "totalNotionalVolume", "dailyNotionalVolume").
    :param exclude_total_data_chart: Boolean flag to exclude the aggregated chart from the response. Default is True.
    :param exclude_total_data_chart_breakdown: Boolean flag to exclude the broken down chart from the response. Default is True.
    :param dex_name: Optional name of the DEX to filter the results. Default is None.
    :return: A dictionary or list containing the summaries of their volumes and dataType history data, filtered by DEX name if provided.
    """
    url = "https://api.llama.fi/overview/options"
    params = {
        "excludeTotalDataChart": str(exclude_total_data_chart).lower(),
        "excludeTotalDataChartBreakdown": str(exclude_total_data_chart_breakdown).lower(),
        "dataType": data_type
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if dex_name:
            filtered_data = [dex for dex in data.get('protocols', []) if dex['name'].lower() == dex_name.lower()]
            return filtered_data
        else:
            return data
    else:
        return {"Error": f"Failed to retrieve data: {response.status_code}"}


get_overview_options_tool = FunctionTool.from_defaults(fn=overview_options)

In [3]:
# from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker

# react_agent = ReActAgent.from_tools(llm=llm, verbose=False, streaming = True)


In [4]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker

react_agent = ReActAgent.from_tools(tools=[fetch_price_tool, internet_search_tool, get_fees_and_revenue_tool,get_tvl_data_tool,get_historicalchain_tvl_data_tool,get_overview_dexs_tool],llm=llm, verbose=True)
# get_pool_tool,fetch_stablecoins_tool,fetch_stable_coin_prices_tool,

In [5]:
while True:
    user_input = input("You : ")
    if user_input.lower() is ['exit' , 'quit' , 'stop' , 'bye']:
        print("Goodbye!")
        break

    response = react_agent.chat(user_input)
    print("Bot : " , response)

> Running step 33661d5c-8477-43e5-b85b-03aa633fcb12. Step input: whats the current price of bitcoin?
Thought: (Implicit) I can answer without any more tools!
Answer: I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.
Bot :  I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.
> Running step 926a20b7-5470-4932-abbb-8cd6610880b8. Step input: get fees and revenue for arbitum protocol.
Thought: (Implicit) I can answer without any more tools!
Answer: I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.
Bot :  I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.
> Running step 74bc5a41-f7c5-41c2-a77f-ed35dea7dec7. Step input: exit
Thought: (Implicit) I can answer without any 

In [5]:
# from playsound import playsound
# def play_audio(filename: str):
#     try:
#         playsound(filename)
#     except Exception as e:
#         print(f"Exception: {e}")
# deepgram = DeepgramClient("85e58b7f3e4d2a466725bce711eb1af1f4cde110")

# async def convert_text_to_speech(text: str, filename: str = "output.mp3"):
#     try:
#         options = SpeakOptions(model="aura-asteria-en")
#         response = await deepgram.asyncspeak.v("1").save(filename, {"text": text}, options)
#         print(f"Audio saved to {filename}")
#     except Exception as e:
#         print(f"Exception: {e}")

# while True:
#     user_input = input("You: ")
#     if user_input.lower() in ['exit', 'quit', 'stop', 'bye']:
#         print("Goodbye!")
#         break

#     response = react_agent.chat(user_input)
#     print("Bot:", response)
#     resp = response.response
#     await convert_text_to_speech(resp)
#     play_audio("output.mp3")

Goodbye!
